In [2]:
import pandas as pd

----
# raw data 만들기
* 로직
    1.  감사의견 없는 연결, 개별, GAAP 합치기
    
        >  연결 -> 개별 -> GAAP 순으로 결측치 대체
    2. 감사의견 있는 연결 파일의 감사의견 컬럼을 새로 만든 통합 데이터프레임에 입력

In [3]:
# 데이터 로드
data_1 = pd.read_csv('./Dataset/외감기업/외감기업(IFRS연결)_감사의견미포함.csv', encoding='euc-kr')
data_2 = pd.read_csv('./Dataset/외감기업/외감기업(IFRS개별).csv', encoding='euc-kr')
data_3 = pd.read_csv('./Dataset/외감기업/외감기업(GAAP).csv', encoding='euc-kr', index_col=0)
data_3.reset_index(drop=True, inplace=True)
data_target = pd.read_csv('./Dataset/외감기업/외감기업(IFRS연결).csv', encoding='euc-kr')

### 데이터 살펴보기

- 데이터 수 : (295975 , 50) + target (295975 , 50)개
- 기업 수   : 51667개

In [4]:
len(data_1['회사명'].unique())

51667

In [5]:
data_1.shape

(295975, 50)

In [6]:
data_2.shape

(295975, 50)

In [7]:
data_3.shape    # 연구비 + 경상개발비

(295975, 51)

In [8]:
data_target.shape  

(295976, 51)

In [9]:
# 컬럼명 저장
col_list = data_1.columns.tolist()

In [10]:
# 연구비, 경상개발비 합치기
data_3['연구비'] = data_3['연구비'] + data_3['경상개발비(요약)(백만원)']
data_3.drop('경상개발비(요약)(백만원)', axis=1, inplace=True)

### 데이터 합치기 전 컬럼명 통일

In [11]:
# 컬럼명 통일
data_1.columns = [f"X_{i}" for i in range(1, 51)]
data_2.columns = [f"X_{i}" for i in range(1, 51)]
data_3.columns = [f"X_{i}" for i in range(1, 51)]

In [12]:
# IFRS연결, IFRS개별 합치기
raw_data_1 = data_1.fillna(data_2)

In [13]:
# IFRS, GAAP 합치기
raw_data_1 = raw_data_1.fillna(data_3)

In [14]:
# 결측치 확인하기
raw_data_1.isnull().sum()

X_1          0
X_2          0
X_3          0
X_4          0
X_5         95
X_6         95
X_7         95
X_8         95
X_9         78
X_10        78
X_11        95
X_12        95
X_13        78
X_14      2151
X_15       348
X_16        78
X_17        78
X_18        78
X_19       122
X_20       103
X_21       103
X_22       103
X_23        78
X_24        94
X_25        94
X_26        94
X_27        94
X_28        94
X_29        94
X_30        94
X_31        93
X_32        93
X_33        93
X_34        93
X_35        93
X_36        93
X_37        93
X_38        93
X_39       107
X_40       107
X_41       107
X_42       107
X_43       107
X_44        78
X_45        78
X_46        78
X_47        78
X_48        78
X_49        83
X_50    271076
dtype: int64

In [15]:
# 다시 원래 컬럼으로 정의하기
raw_data_1.columns = col_list

- data_target이 feature 데이터 보다 하나의 콜럼이 많은 이유!!
    * 중복되는 하나의 행이 있었다
    * 에이케이에스앤디주식회사(2012)행 삭제!!!

In [16]:
error_raw = data_target[data_target.duplicated(keep=False)]
error_raw.index

Index([130263, 130264], dtype='int64')

In [17]:
data_target=data_target.drop(130263,axis=0).reset_index(drop=True)

In [18]:
data_target.shape

(295975, 51)

In [19]:
raw_data_1.shape

(295975, 50)

---

- 합치기 전 검증!!!

In [20]:
(raw_data_1['회사명']==data_target['회사명']).value_counts()

회사명
True    295975
Name: count, dtype: int64

In [21]:
(raw_data_1['회계년도']==data_target['회계년도']).value_counts()

회계년도
True    295975
Name: count, dtype: int64

In [22]:
raw_data_2=pd.concat([raw_data_1,data_target.iloc[:,-1]],axis=1)
raw_data_2

,회사명,거래소코드,회계년도,통계청 한국표준산업분류 코드 10차(대분류),부채비율,당좌비율,유동비율,이자보상배율(이자비용),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),이자비용(IFRS연결)(천원),...,자본분배율,이윤분배율,투자활동으로 인한 현금흐름(*)(IFRS연결)(천원),재무활동으로 인한 현금흐름(*)(IFRS연결)(천원),자산(*)(IFRS연결)(천원),부채(*)(IFRS연결)(천원),자본(*)(IFRS연결)(천원),연구개발비(IFRS연결)(천원),* (정상)영업손익(보고서기재)(IFRS연결)(천원),감사의견코드
0,(유)가든주류,97082,2019/12,46,178.34,213.01,243.06,3.17,138498.0,628271.0,...,47.99,22.12,-1972954.0,1982120.0,33246572.0,21302019.0,11944553.0,1992.0,NaN,NaN
1,(유)가든주류,97082,2020/12,46,361.81,115.89,128.39,1.29,-341826.0,476716.0,...,27.92,8.72,-914119.0,892418.0,32940746.0,25807747.0,7132998.0,615.0,NaN,QS
2,(유)가든주류,97082,2021/12,46,281.83,176.11,186.39,0.29,130657.0,476252.0,...,34.33,13.86,4066202.0,-4287909.0,29500238.0,21774189.0,7726049.0,137.0,NaN,UQ
3,(유)가든주류,97082,2022/12,46,0.00,28.33,35.17,-1.44,-26425950.0,1914030.0,...,-167.60,-397.51,13539698.0,13075530.0,9861648.0,34820255.0,-24958607.0,-2749.0,NaN,DU
4,(유)그룹세브코리아,13171,2019/12,46,147.02,92.20,158.78,25.79,14297932.0,468007.0,...,67.67,36.71,-718431.0,-14582988.0,50917276.0,30304375.0,20612901.0,12070.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295970,힐티코리아주식회사,10534,2021/12,46,83.55,116.00,204.04,16.92,1399525.0,170487.0,...,24.71,8.53,-580519.0,-5714997.0,57962904.0,26384600.0,31578304.0,0.0,2884587.0,UQ
295971,힐티코리아주식회사,10534,2022/12,46,108.21,95.88,182.33,45.26,8906854.0,173153.0,...,35.23,18.72,-621622.0,-2228349.0,80263333.0,41714240.0,38549093.0,0.0,7837661.0,UQ
295972,（주）신미코스메틱,85105,2017/12,46,31.40,416.15,416.15,19368.35,13717057.0,1309.0,...,98.51,76.19,-2137038.0,-227960.0,27238719.0,6508768.0,20729950.0,25353.0,NaN,NaN
295973,（주）신미코스메틱,85105,2018/12,46,15.51,677.00,755.04,7918.22,18445523.0,2475.0,...,95.14,59.64,-15323100.0,-1574040.0,30667235.0,4117284.0,26549952.0,19598.0,NaN,UQ


In [23]:
raw_data_2.describe()

,거래소코드,통계청 한국표준산업분류 코드 10차(대분류),부채비율,당좌비율,유동비율,이자보상배율(이자비용),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),이자비용(IFRS연결)(천원),차입금의존도,자기자본구성비율,...,노동소득분배율,자본분배율,이윤분배율,투자활동으로 인한 현금흐름(*)(IFRS연결)(천원),재무활동으로 인한 현금흐름(*)(IFRS연결)(천원),자산(*)(IFRS연결)(천원),부채(*)(IFRS연결)(천원),자본(*)(IFRS연결)(천원),연구개발비(IFRS연결)(천원),* (정상)영업손익(보고서기재)(IFRS연결)(천원)
count,295975.000000,295975.000000,2.958800e+05,2.958800e+05,2.958800e+05,2.958800e+05,2.958970e+05,2.958970e+05,2.958800e+05,295880.000000,...,2.958680e+05,2.958680e+05,2.958680e+05,2.958970e+05,2.958970e+05,2.958970e+05,2.958970e+05,2.958970e+05,2.958920e+05,2.489900e+04
mean,62103.418167,43.671864,2.163986e+04,1.560940e+04,1.649745e+04,1.181900e+08,3.066959e+06,9.917524e+05,2.158403e+02,36.034839,...,8.934938e+01,-7.649434e+00,-2.446380e+04,-4.448067e+06,1.869084e+06,7.290401e+07,4.445242e+07,2.845159e+07,1.792640e+05,1.539469e+07
std,25035.184135,20.469855,6.142931e+06,4.494784e+06,4.501424e+06,3.228222e+08,6.716188e+07,6.417346e+06,3.511742e+04,32.273486,...,1.207413e+04,1.207405e+04,8.054248e+06,6.726894e+07,4.566440e+07,6.211577e+08,3.269636e+08,3.490838e+08,1.286059e+07,1.215514e+08
min,10000.000000,1.000000,-6.280000e+00,-3.100000e+00,-3.100000e+00,-5.124345e+08,-3.450823e+09,-3.806243e+06,0.000000e+00,-8444.950000,...,-7.829730e+03,-6.381824e+06,-2.945443e+09,-1.479378e+10,-5.082106e+09,-4.061700e+04,-9.535820e+05,-2.716877e+09,-8.552200e+05,-1.936136e+09
25%,46303.000000,26.000000,3.319750e+01,3.296000e+01,6.120000e+01,0.000000e+00,-4.731180e+05,5.926300e+04,1.204000e+01,11.390000,...,7.040000e+00,8.570000e+00,0.000000e+00,-2.672741e+06,-1.023304e+06,1.353553e+07,7.920664e+06,1.832628e+06,-2.700000e+01,-4.950690e+05
50%,64451.000000,42.000000,1.330300e+02,7.954000e+01,1.142000e+02,2.350000e+00,6.315700e+05,2.443350e+05,4.048000e+01,32.120000,...,2.919000e+01,5.565000e+01,1.422000e+01,-5.176800e+05,0.000000e+00,2.134103e+07,1.302122e+07,6.838031e+06,5.820000e+02,1.685420e+06
75%,81220.500000,68.000000,3.418700e+02,1.631900e+02,2.140800e+02,1.830250e+01,2.646083e+06,5.726590e+05,6.485000e+01,56.782500,...,5.117000e+01,7.596250e+01,3.852000e+01,0.000000e+00,1.759162e+06,4.276756e+07,2.619102e+07,1.669974e+07,2.092000e+03,7.099280e+06
max,107278.000000,98.000000,2.373601e+09,2.030502e+09,2.030502e+09,1.733808e+09,1.139583e+10,5.937832e+08,1.437589e+07,106.700000,...,6.381924e+06,7.929730e+03,3.509009e+08,2.836892e+09,8.973336e+09,6.953855e+10,4.325749e+10,5.763866e+10,2.855357e+09,5.883246e+09
